In [1]:
import pandas as pd
import numpy as np

In [2]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', attrs={'class': 'wikitable'}, header=0)

In [3]:
df = dfs[0]

In [4]:
df = df[df.Borough!="Not assigned"]

In [5]:
df = df.groupby(['Postcode', 'Borough'],sort=False, as_index=False).agg(','.join)

In [6]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'],df['Neighbourhood'])

In [7]:
df.shape

(103, 3)

Using csv module to read the coordinates file \[geocoder didn't work\]

In [11]:
import csv
with open('Geospatial_Coordinates.csv') as file:
    coordinates = list(csv.DictReader(file))

coordinates[:5]

[OrderedDict([('Postal Code', 'M1B'),
              ('Latitude', '43.8066863'),
              ('Longitude', '-79.1943534')]),
 OrderedDict([('Postal Code', 'M1C'),
              ('Latitude', '43.7845351'),
              ('Longitude', '-79.1604971')]),
 OrderedDict([('Postal Code', 'M1E'),
              ('Latitude', '43.7635726'),
              ('Longitude', '-79.1887115')]),
 OrderedDict([('Postal Code', 'M1G'),
              ('Latitude', '43.7709921'),
              ('Longitude', '-79.2169174')]),
 OrderedDict([('Postal Code', 'M1H'),
              ('Latitude', '43.773136'),
              ('Longitude', '-79.2394761')])]

Map longitude and latitude with postcodes

In [14]:
lngs = []
lats = []
for postcode in df['Postcode']:
    for coordinate in coordinates:
        if postcode == coordinate['Postal Code']:
            lngs.append(coordinate['Longitude'])
            lats.append(coordinate['Latitude'])
    

In [15]:
df['Latitude'] = lats
df['Longitude'] = lngs
df[:10]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938
5,M9A,Etobicoke,Islington Avenue,43.6678556,-79.5322424
6,M1B,Scarborough,"Rouge,Malvern",43.8066863,-79.1943534
7,M3B,North York,Don Mills North,43.7459058,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.7063972,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.6571618,-79.3789371


In [19]:
from geopy.geocoders import Nominatim

address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [21]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto